# Segment SDS Archive based on launch event times

Launch times come from 'PilotStudy_KSC_Rocket_Launches.xlsx'
SDS archives are at D:/ROCKETSEIS/SDS and contain data from wells 6I and 6S and from seismo-acoustic stations
Segmented event waveform files are saved as MiniSEED to D:/Dropbox/DATA/KSC/KSC_Well_Seismoacoustic_Data/Events

## 1. Read launch data into a DataFrame and generate a list of launch times in Python datetime.datetime format

In [1]:
%run header.ipynb
configDict = wrappers.read_config(configdir=paths['CONFIGDIR'], leader='KSC')
SDS_TOP = configDict['general']['SDS_TOP']

DROPBOX_PROJECT_DIR = os.path.join(paths['DROPBOX_TOP'], 'PROFESSIONAL', 'RESEARCH', '3_Project_Documents', 'NASAprojects', '201602 Rocket Seismology', '202010 KSC Launchpad Erosion')
RAWDATA_DIR = os.path.join(paths['DROPBOX_TOP'], 'DATA', 'KSC', 'KSC_Well_Seismoacoustic_Data')
EVENT_MSEED_DIR = os.path.join(RAWDATA_DIR, 'Events')
#HTML_DIR = './html'
#PLOTS_DIR = HTML_DIR
excel_launches = os.path.join(DROPBOX_PROJECT_DIR, 'PilotStudy_KSC_Rocket_Launches.xlsx')
csv_launches = excel_launches.replace('.xlsx', '.csv')
#launchesDF = pd.read_excel(excel_launches, sheet_name='launches') # header=0, names=None, index_col=None, usecols=None, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=_NoDefault.no_default, date_format=None, thousands=None, decimal='.', comment=None, skipfooter=0, storage_options=None, dtype_backend=_NoDefault.no_default, engine_kwargs=None)
launchesDF = pd.read_csv(csv_launches)
dt_tmp = pd.to_datetime(launchesDF['Date'] + ' ' +  launchesDF['Time'])
launchesDF['Date'] = [pdt.to_pydatetime() for pdt in dt_tmp]
launchesDF.drop(labels='Time', axis=1, inplace=True)
#print(launchesDF)

## 2. For each launch, segment raw SDS data to multi-trace MiniSEED file in Events directory

In [2]:
#import matplotlib.pyplot as plt

def event_sds2miniseed(launchtime, thisSDSobj, EVENT_MSEED_DIR):    
    pretrig = 600
    posttrig = 1200
    mseedfile = os.path.join(EVENT_MSEED_DIR, 'launch_%s_raw.mseed' % launchtime.strftime('%Y%m%dT%H%M%S'))
    if os.path.exists(mseedfile):
        print('%s already exists' % mseedfile)
    else:
        print('%s not found. Segmenting from SDS.' % mseedfile) 
        startt = launchtime - pretrig
        endt = launchtime + posttrig
        thisSDSobj.read(startt, endt, speed=1)
        st = thisSDSobj.stream
        if len(st)>0:
            st.write(mseedfile, 'mseed')
        else:
            print('Got no data')
            mseedfile = None
    return mseedfile

def clean(st):
    for tr in st:
        if tr.stats.network != 'FL':
            continue
        tr.detrend('linear')
        tr.filter('highpass', freq=0.01, corners=2) 
        
def apply_calibration_correction(st):
    # calibration correction

    for tr in st:
        if 'countsPerUnit' in tr.stats:
            continue
        else:
            tr.stats['countsPerUnit'] = 0.0  
            if tr.stats.channel[1]=='D':
                tr.stats.countsPerUnit = 720 # counts/Pa on 1 V FS setting
                if tr.id[:-1] == 'FL.BCHH3.10.HD':
                    if tr.stats.starttime < UTCDateTime(2022,5,26): # Chaparral M25. I had it set to 1 V FS. Should have used 40 V FS. 
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 8e5 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 720 # counts/Pa 
                    else: # Chaparral switched to 40 V FS
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 2e4 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 18 # counts/Pa 
                tr.stats.units = 'Pa'

            elif tr.stats.channel[1]=='H':
                tr.stats.countsPerUnit = 3e8 # counts/(m/s)
                tr.stats.units = 'm/s'
            tr.data = tr.data/tr.stats.countsPerUnit
    


def maxamp(tr):
    return np.max(np.abs(tr.data))

def remove_spikes(st):
    SEISMIC_MAX = 0.1 # m/s
    INFRASOUND_MAX = 3000 # Pa
    SEISMIC_MIN = 1e-9
    INFRASOUND_MIN = 0.01
    
    for tr in st:
        ma = maxamp(tr)
        if tr.stats.units == 'm/s':
            tr.data[tr.data > SEISMIC_MAX] = np.nan
            tr.data[tr.data < -1 * SEISMIC_MAX] = np.nan             
        elif tr.stats.units == 'Pa':
            tr.data[tr.data > INFRASOUND_MAX] = np.nan
            tr.data[tr.data < -1 * INFRASOUND_MAX] = np.nan     

from obspy.signal.trigger import coincidence_trigger
from pprint import pprint
import matplotlib.dates as dates
def detectEvent(st, launchtime):
    trig = coincidence_trigger("recstalta", 3.5, 1, st, 3, sta=2, lta=40)
    best_trig = {}
    best_product = 0
    for this_trig in trig:
        thistime = dates.date2num(this_trig['time'])
        this_product = this_trig['coincidence_sum']*this_trig['duration']
        if this_product > best_product:
            best_trig = this_trig
            best_product = this_product
    pprint(best_trig)
    return best_trig['time']


def add_snr(st, assoctime, threshold=1.5):
    nstime = max([st[0].stats.starttime, assoctime-240])
    netime = min([st[0].stats.endtime, assoctime-60])
    sstime = assoctime
    setime = min([st[0].stats.endtime, assoctime+120])    
    for tr in st:
        tr_noise = tr.copy().trim(starttime=nstime, endtime=netime)
        tr_signal = tr.copy().trim(starttime=sstime, endtime=setime)
        tr.stats['noise'] = np.nanmedian(np.abs(tr_noise.data))
        tr.stats['signal'] = np.nanmedian(np.abs(tr_signal.data))
        tr.stats['snr'] = tr.stats['signal']/tr.stats['noise']



thisSDSobj = SDS.SDSobj(SDS_TOP)

if not 'mseedfile' in launchesDF.columns:
    launchesDF['mseedfile'] = None
if not 'detected' in launchesDF.columns:
    launchesDF['detected'] = None                             
for i, row in launchesDF.iterrows():
    launchTime = UTCDateTime(row['Date'])
    print('Processing launch at %s' % launchTime.strftime('%Y-%m-%d %H:%M:%S'))                         
    if not row['detected']:
        if not row['mseedfile']:
            mseedfile = event_sds2miniseed(launchTime, thisSDSobj, EVENT_MSEED_DIR)
            launchesDF.at[i, 'mseedfile'] = mseedfile
        try:
            st = read(mseedfile)    
            st.merge()
        except:
            st = Stream()
        print('%s: %d channels' % (mseedfile,len(st)))
        
        # Now we can clean, correct, despike, and detect
        SA = Stream()
        for tr in st:
            if tr.stats.network[0] != '6':
                SA.append(tr)
        if len(SA)==0:
            continue
        
        clean(SA)
        apply_calibration_correction(SA)
        remove_spikes(SA)
        print('- %d channels after despiking' % len(SA))
        assocTime = detectEvent(SA, launchTime)

        #look_for_signal(SA, assocTime, threshold=1.1)
        detectedfile =  os.path.join(EVENT_MSEED_DIR, 'launch_%s_detected.pkl' % launchTime.strftime('%Y%m%dT%H%M%S'))
        print('Writing %s' % detectedfile)
        st.write(detectedfile, format='PICKLE')  
        launchesDF.at[i, 'detected'] = assocTime
            
        # loop over stations, plotting all traces
        stations = list(set([tr.stats.station for tr in st]))
        for station in stations:
            station_stream = st.copy().select(station=station)
            station_stream.trim(starttime=assocTime-60, endtime=assocTime+200).plot(equal_scale=False, outfile=detectedfile.replace('.pkl', '_%s.png' % station))
del thisSDSobj
launchesDF.to_csv(csv_launches.replace('.csv','_detected.csv'))          

Processing launch at 2022-04-01 16:24:16
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220401T162416_raw.mseed not found. Segmenting from SDS.
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD4.D\FL.BCHH2.10.HD4.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD5.D\FL.BCHH2.10.HD5.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD6.D\FL.BCHH2.10.HD6.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD7.D\FL.BCHH2.10.HD7.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD8.D\FL.BCHH2.10.HD8.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD9.D\FL.BCHH2.10.HD9.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HHE.D\FL.BCHH3.00.HHE.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HHN.D\FL.BCHH3.00.HHN.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HHZ.D\FL.BCHH3.00.HHZ.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HDF.D\FL.BCHH3.10.HDF.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A1\HHE.D\FL.S39A1.

st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A2\HHN.D\FL.S39A2.00.HHN.D.2022.111")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A2\HHZ.D\FL.S39A2.00.HHZ.D.2022.111")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A2\HDF.D\FL.S39A2.10.HDF.D.2022.111")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HD2.D\FL.S39A3.00.HD2.D.2022.111")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HD3.D\FL.S39A3.00.HD3.D.2022.111")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HDF.D\FL.S39A3.00.HDF.D.2022.111")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HD2.D\FL.S39A3.10.HD2.D.2022.111")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HD3.D\FL.S39A3.10.HD3.D.2022.111")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HDF.D\FL.S39A3.10.HDF.D.2022.111")
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220421T175140_raw.mseed: 24 channels
- 24 channels after despiking
{'coincidence_sum': 18.0,
 'duration': 110.76999998092651,
 'similarity': {},
 'stations': ['S39A2',
              'BCHH2',
              'BCHH2',
              'BCHH

Processing launch at 2022-05-06 09:42:00
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220506T094200_raw.mseed not found. Segmenting from SDS.
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD4.D\FL.BCHH2.10.HD4.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD5.D\FL.BCHH2.10.HD5.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD6.D\FL.BCHH2.10.HD6.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD7.D\FL.BCHH2.10.HD7.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD8.D\FL.BCHH2.10.HD8.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD9.D\FL.BCHH2.10.HD9.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HHE.D\FL.BCHH3.00.HHE.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HHN.D\FL.BCHH3.00.HHN.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HHZ.D\FL.BCHH3.00.HHZ.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HDF.D\FL.BCHH3.10.HDF.D.2022.126")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A1\HHE.D\FL.S39A1.

st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A2\HHN.D\FL.S39A2.00.HHN.D.2022.138")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A2\HHZ.D\FL.S39A2.00.HHZ.D.2022.138")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A2\HDF.D\FL.S39A2.10.HDF.D.2022.138")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HD2.D\FL.S39A3.00.HD2.D.2022.138")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HD3.D\FL.S39A3.00.HD3.D.2022.138")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HDF.D\FL.S39A3.00.HDF.D.2022.138")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HD2.D\FL.S39A3.10.HD2.D.2022.138")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HD3.D\FL.S39A3.10.HD3.D.2022.138")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A3\HDF.D\FL.S39A3.10.HDF.D.2022.138")
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220518T105940_raw.mseed: 24 channels
- 24 channels after despiking
{'coincidence_sum': 19.0,
 'duration': 122.44999980926514,
 'similarity': {},
 'stations': ['S39A2',
              'S39A1',
              'S39A2',
              'S39A

Processing launch at 2022-06-08 21:04:00
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220608T210400_raw.mseed not found. Segmenting from SDS.
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD4.D\FL.BCHH2.10.HD4.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD5.D\FL.BCHH2.10.HD5.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD6.D\FL.BCHH2.10.HD6.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD7.D\FL.BCHH2.10.HD7.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD8.D\FL.BCHH2.10.HD8.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD9.D\FL.BCHH2.10.HD9.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH4\HD2.D\FL.BCHH4.00.HD2.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH4\HD3.D\FL.BCHH4.00.HD3.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH4\HDF.D\FL.BCHH4.00.HDF.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH4\HHE.D\FL.BCHH4.10.HHE.D.2022.159")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH4\HHN.D\FL.BCHH4.

...

D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20221101T134100_raw.mseed: 30 channels
- 20 channels after despiking
{'coincidence_sum': 18.0,
 'duration': 85.1399998664856,
 'similarity': {},
 'stations': ['S39A1',
              'S39A2',
              'S39A1',
              'S39A2',
              'S39A1',
              'S39A2',
              'S39A1',
              'S39A2',
              'BCHH4',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'BCHH4'],
 'time': UTCDateTime(2022, 11, 1, 13, 40, 43, 640000),
 'trace_ids': ['FL.S39A1.00.HHE',
               'FL.S39A2.00.HHN',
               'FL.S39A1.00.HHN',
               'FL.S39A2.00.HHE',
               'FL.S39A1.00.HHZ',
               'FL.S39A2.00.HHZ',
               'FL.S39A1.10.HDF',
               'FL.S39A2.10.HDF',
               'FL.BCHH4.10.HHZ',
               '

C:\Users\thompsong\Anaconda3\lib\site-packages\obspy\io\mseed\core.py:838: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')
C:\Users\thompsong\Anaconda3\lib\site-packages\obspy\io\mseed\core.py:840: UserWarning: File will be written with more than one different record lengths.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'record lengths')


D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20221103T052200_raw.mseed: 30 channels
- 20 channels after despiking
{'coincidence_sum': 15.0,
 'duration': 67.94000005722046,
 'similarity': {},
 'stations': ['BCHH2',
              'BCHH4',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'S39A1',
              'BCHH4',
              'S39A1',
              'S39A1',
              'BCHH4',
              'S39A1'],
 'time': UTCDateTime(2022, 11, 3, 5, 21, 59, 580000),
 'trace_ids': ['FL.BCHH2.10.HD4',
               'FL.BCHH4.10.HHN',
               'FL.BCHH4.10.HHZ',
               'FL.BCHH2.10.HD7',
               'FL.BCHH2.10.HD6',
               'FL.BCHH4.00.HDF',
               'FL.BCHH2.10.HD8',
               'FL.BCHH2.10.HD9',
               'FL.BCHH4.00.HD2',
               'FL.S39A1.00.HHE',
               'FL.BCHH4.10.HHE',
               'F

C:\Users\thompsong\Anaconda3\lib\site-packages\obspy\io\mseed\core.py:838: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')
C:\Users\thompsong\Anaconda3\lib\site-packages\obspy\io\mseed\core.py:840: UserWarning: File will be written with more than one different record lengths.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'record lengths')


D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20221116T064744_raw.mseed: 14 channels
- 4 channels after despiking
{'coincidence_sum': 4.0,
 'duration': 51.6199996471405,
 'similarity': {},
 'stations': ['S39A1', 'S39A1', 'S39A1', 'S39A1'],
 'time': UTCDateTime(2022, 11, 16, 6, 47, 41, 350000),
 'trace_ids': ['FL.S39A1.00.HHE',
               'FL.S39A1.00.HHN',
               'FL.S39A1.00.HHZ',
               'FL.S39A1.10.HDF']}
Writing D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20221116T064744_detected.pkl
Processing launch at 2022-11-23 02:57:00
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20221123T025700_raw.mseed not found. Segmenting from SDS.
st = read("D:/ROCKETSEIS/SDS\2022\6I\0XXXX\HDH.D\6I.0XXXX.XX.HDH.D.2022.327")
st = read("D:/ROCKETSEIS/SDS\2022\6I\12264\HDD.D\6I.12264.29.HDD.D.2022.327")
st = read("D:/ROCKETSEIS/SDS\2022\6I\21498\HDD.D\6I.21498.82.HDD.D.2022.327")
st = read("D:/ROCKETSEIS/SDS\2022\6I\21516\HDD.D\6I.215

## 3. For seismo-acoustic traces, clean data, calibrate, remove spikes, run network detection, save new MiniSEED files

In [ ]:
def load_event(launchtime):            
    mseedfile = time_TO_miniseed_filename(launchtime)
    try:
        st = read(mseedfile)    
        st.merge()
    except:
        st = Stream()
    print('%s: %d channels' % (mseedfile,len(st)))
    return st

def clean(st):
    for tr in st:
        if tr.stats.network != 'FL':
            continue
        tr.detrend('linear')
        tr.filter('highpass', freq=0.01, corners=2)    

def apply_calibration_correction(straw):
    # calibration correction
    stcorrected = straw.copy()
    for tr in stcorrected:
        if tr.stats.network != 'FL':
            continue
        if 'countsPerUnit' in tr.stats:
            continue
        else:
            tr.stats['countsPerUnit'] = 0.0  
            if tr.stats.channel[1]=='D':
                tr.stats.countsPerUnit = 720 # counts/Pa on 1 V FS setting
                if tr.id[:-1] == 'FL.BCHH3.10.HD':
                    if tr.stats.starttime < UTCDateTime(2022,5,26): # Chaparral M25. I had it set to 1 V FS. Should have used 40 V FS. 
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 8e5 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 720 # counts/Pa 
                    else: # Chaparral switched to 40 V FS
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 2e4 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 18 # counts/Pa 
                tr.stats.units = 'Pa'

            elif tr.stats.channel[1]=='H':
                tr.stats.countsPerUnit = 3e8 # counts/(m/s)
                tr.stats.units = 'm/s'
            tr.data = tr.data/tr.stats.countsPerUnit
        #print(tr.id, tr.stats.countsPerUnit)
    return stcorrected        


def maxamp(tr):
    return np.max(np.abs(tr.data))

def remove_spikes(st):
    SEISMIC_MAX = 0.1 # m/s
    INFRASOUND_MAX = 3000 # Pa
    SEISMIC_MIN = 1e-8
    INFRASOUND_MIN = 0.01
    st2 = Stream()
    
    for tr in st:
        ma = maxamp(tr)
        if tr.stats.units == 'm/s':
            if ma < SEISMIC_MAX and ma > SEISMIC_MIN:
                st2.append(tr)
            else:
                print('Not appending %s. Max value %e m/s out of range' % (tr.id, ma))               
        elif tr.stats.units == 'Pa':
            if ma < INFRASOUND_MAX  and ma > INFRASOUND_MIN:
                st2.append(tr)    
            else:
                print('Not appending %s. Max value %e Pa out of range' % (tr.id, ma))
    return st2 

from obspy.signal.trigger import coincidence_trigger
from pprint import pprint
import matplotlib.dates as dates
def detectEvent(st, launchtime):
    trig = coincidence_trigger("recstalta", 3.5, 1, st, 3, sta=2, lta=40)
    best_trig = {}
    best_product = 0
    for this_trig in trig:
        thistime = dates.date2num(this_trig['time'])
        this_product = this_trig['coincidence_sum']*this_trig['duration']
        if this_product > best_product:
            best_trig = this_trig
            best_product = this_product
    pprint(best_trig)
    st_detected = Stream()
    for tr in st:
        if tr.id in best_trig['trace_ids']:
            st_detected.append(tr)
    #if len(st_detected)>0:
    #    st_detected.plot(equal_scale=False, starttime=best_trig['time']-60, endtime=best_trig['time']+120 )
    #input("ENTER to continue")
    return st_detected, best_trig['time']

import numpy as np
def look_for_signal(st, launchtime, threshold=1.5):
    st2 = Stream()
    #clean(st)
    for tr in st:
        tr_noise = tr.copy().trim(starttime=launchtime-240, endtime=launchtime-120)
        tr_signal = tr.copy().trim(starttime=launchtime, endtime=launchtime+120)
        avnoise = np.nanmedian(np.abs(tr_noise.data))
        avsignal = np.nanmedian(np.abs(tr_signal.data))
        #print(avsignal, avnoise)
        if avsignal > threshold * avnoise:
            st2.append(tr)
    return st2



assocTimes = []
for launch_datetime in launch_times:
    launchTime = UTCDateTime(launch_datetime)
    print('Processing launch at %s' % launchTime.strftime('%Y-%m-%d %H:%M:%S'))
    st = load_event(launchTime)
    if len(st)==0:
        assocTimes.append(None)
        continue
    print(st)
    
    SA = st.select(network='FL')
    clean(SA)
    print('- %d channels after cleaning' % len(SA))
    
    SAc = apply_calibration_correction(SA)
    print('- %d channels after calibrating' % len(SAc))
    
    # SCAFFOLD: alter this to applied thresholds
    SAcd = remove_spikes(SAc)
    print('- %d channels after despiking' % len(SAcd))
    if len(SAcd)>0:
        mseedfile = time_TO_miniseed_filename(launchTime, raw=False, calibrated=True)
        print('Writing %s' % mseedfile)
        SAcd.write(mseedfile)
    
    # SCAFFOLD: alter this to detect/assoc events & correct launchtime
    SAcda, assocTime= detectEvent(SAcd, launchTime)
    assocTimes.append(assocTime)
    print('- %d channels after detection' % len(SAcda))
    
    SAcda_snr = look_for_signal(SAcda, assocTime, threshold=1.1)
    print('- %d channels after SNR check' % len(SAcda_snr))
    
    if len(SAcda_snr)>0:
        mseedfile = time_TO_miniseed_filename(assocTime, raw=False, detected=True)
        print('Writing %s' % mseedfile)
        SAcda_snr.write(mseedfile)
        

print(assocTimes)
launchesDF['assocTime'] = assocTimes

In [9]:
HTML_DIR = os.path.join(RAWDATA_DIR, 'html')
indexfile = open(os.path.join(HTML_DIR, "index.html"), "w")
indexfile.write("""
<html>

<head>
<title>Launch data browser</title>
</head>

<body>

<table border=1>

<tr>
<th>Detected launch time</th>
<th>JulDay</th>
<th>epoch</th>
<th>Seismic</th>
<th>Infrasound</th>
<th>Well_6S</th>
<th>Well_6I</th>
</tr>

<tr>
""")


for i, row in launchesDF.iterrows():
    print('\n\n\n')
    launchTime = UTCDateTime(row['Date'])
    print('Processing launch at %s' % launchTime.strftime('%Y-%m-%d %H:%M:%S'))  
    indexfile.write("<td>%s</td>\n<td>%s</td>\n<td>%.0f</td>\n" % (launchTime, launchTime.julday, launchTime.timestamp))
    
    ## Seismic
    detectedfile =  os.path.join(EVENT_MSEED_DIR, 'launch_%s_detected.pkl' % launchTime.strftime('%Y%m%dT%H%M%S'))
    seismicpngfile = os.path.join(HTML_DIR, 'launch_%s_seismic.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    if not os.path.exists(seismicpngfile):
        print('Reading %s' % detectedfile)
        try:
            st = read(detectedfile, format='PICKLE')
            if row['detected']:
                dt = row['detected']
                stime = max([st[0].stats.starttime, dt-30])
                etime = min([st[0].stats.endtime, dt+150])
                st.trim(starttime=stime, endtime=etime)
        except:
            st = Stream()
        st_seismic = st.select(channel='HH?').select(network='FL')
        if len(st_seismic)>0:
            st_seismic.plot(equal_scale=False, outfile=seismicpngfile )            
    if os.path.exists(seismicpngfile):
        indexfile.write("<td><a href=\'%s\'>%d</a></td>\n" % (os.path.basename(seismicpngfile), len(st_seismic)))
    else:
        indexfile.write("<td>0</td>\n")
        
    ## Acoustic
    acousticpngfile = os.path.join(HTML_DIR, 'launch_%s_infrasound.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    if not os.path.exists(acousticpngfile):
        if not st: # in locals():
            print('Reading %s' % detectedfile)
            try:
                st = read(detectedfile, format='PICKLE')
            except:
                st = Stream()
        st_acoustic = st.select(channel='HD?').select(network='FL')
        
        if len(st_acoustic)>0: 
            st_acoustic.plot(equal_scale=False, outfile=acousticpngfile )
    if os.path.exists(acousticpngfile):        
        indexfile.write("<td><a href=\'%s\'>%d</a></td>\n" % (os.path.basename(acousticpngfile), len(st_acoustic) ))
    else:
        indexfile.write("<td>0</td>\n")    
         
    # Well 6S
    for well in ['6S', '6I']:
        wellpngfile = os.path.join(HTML_DIR, 'launch_%s_well_%s.png' % (launchTime.strftime('%Y%m%dT%H%M%S'), well) )
        if not os.path.exists(wellpngfile):
            if not 'st': # in locals():
                print('Reading %s' % detectedfile)
                try:
                    st = obspy.read(detectedfile)
                except:
                    st = Stream()
            st_well = st.select(network=well)
            if len(st_well)>0: 
                st_well.plot(equal_scale=False, outfile=wellpngfile )
        if os.path.exists(wellpngfile):        
            indexfile.write("<td><a href=\'%s\'>%d</a></td>\n" % (os.path.basename(wellpngfile), len(st_well)))
        else:
            indexfile.write("<td>0</td>\n")              

        
    # comments
    #indexfile.write("<td>%s</td>\n" % launchcomments[launchnum])
    
    # end this row of table
    indexfile.write("</tr>\n\n")

indexfile.write("""

</table>

</body>

</html>
""")
indexfile.close()






Processing launch at 2022-04-01 16:24:16
Reading D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220401T162416_detected.pkl




Processing launch at 2022-04-08 15:17:12
Reading D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220408T151712_detected.pkl




Processing launch at 2022-04-21 17:51:40
Reading D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220421T175140_detected.pkl




Processing launch at 2022-04-27 07:52:55
Reading D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220427T075255_detected.pkl




Processing launch at 2022-04-29 21:27:10
Reading D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220429T212710_detected.pkl




Processing launch at 2022-05-06 09:42:00
Reading D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220506T094200_detected.pkl




Processing launch at 2022-05-14 20:40:50
Reading D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220514T20405

In [24]:
import glob
launchpngs = sorted(glob.glob(os.path.join(HTML_DIR, "*_seismic.png")))


    
htmlstr = '''
<html>
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Launch Events</title>

    <!-- Basic style -->
    <style>
        .img-container {
            margin: 20px auto;
            width: 40%%;
        }

        .btn-container {
            text-align: center;
        }

        img {
            height: 600px;
            width: 600px;
            display: block;
            margin: auto;
            padding-bottom: 20px;
        }

        button {
            outline: none;
            padding: 12px;
            border: none;
            background-color: rgb(25, 121, 211);
            border-radius: 8px;
            color: white;
            cursor: pointer;
        }
    </style>

</head>
<body>
    <!-- HTML code for next and previous button -->
    <div class="img-container">
'''
htmlstr += f"<img src='{launchpngs[0]}' class='imageTag'>"
htmlstr += '''
        <div class="btn-container">
            <button onclick="previous()">Previous</button>
            <button onclick="next()">Next</button>
        </div>
    </div>

    <!-- Javascript code for next and previous button -->
    <script>
        let images = [
'''
for launchpng in launchpngs:
    htmlstr += f"'{launchpng}',"
htmlstr += '''
        ];
        let imageTag = document.querySelector('.imageTag');
        let i = 0;

        function next() {
            if (i >= images.length - 1) {
                return false;
            }
            i++;
            imageTag.setAttribute('src', images[i]);
        }
        function previous() {
            if (i <= 0) {
                return false;
            }
            i--;
            imageTag.setAttribute('src', images[i]);
        }

    </script>

</body>

</html>
'''

with open(os.path.join(HTML_DIR,"launches.html"), "w") as text_file:
    text_file.write(htmlstr)